In [1]:
#make mongodb

In [6]:
from mongoengine import connect
connect('cnf')
from mongo_models import (
    CNFFoodGroup, CNFFoodSource, CNFNutrientSource, 
    CNFRefuseName, CNFYieldName, CNFMeasureName,
    CNFFoodName, CNFConversionFactor, CNFNutrientAmount,
    CNFYieldAmount, CNFRefuseAmount, CNFNutrientName
)


In [ ]:
#https://github.com/cyrilrbt/canadian-nutrient-file/blob/master/cnf/scripts.py

In [21]:
import os
import csv
import time
#from flask_script import Command, Option
#from cnf.settings import ROOT_PATH

data_path = 'cnf_csv'


def csv_reader(filename):
    print(f' filepath: {os.path.join(data_path, filename)}')
    _f = open(os.path.join(data_path, filename, ), 'r', encoding='iso8859')
    r = csv.reader(_f)
    next(r)
    return r

def load_data(iterable, model, field_map, batch=1000):
    total = 0
    pending = []
    for items in iterable:
        items = [x for x in items if x]
        if items:
            pending.append(model(**dict(zip(field_map, items))))
            total += 1
            if len(pending) >= batch:
                model.objects.insert(pending)
                pending = []
    if pending:
        model.objects.insert(pending)
    return total

def run(source, batch):
    batch = int(batch)
    data_path = source
    #from flask import current_app as app
    
    from mongo_models import (
        CNFFoodGroup, CNFFoodSource, CNFFoodName, CNFNutrientAmount,
        CNFNutrientName, CNFNutrientSource, CNFRefuseAmount, CNFRefuseName,
        CNFYieldAmount, CNFYieldName, CNFConversionFactor, CNFMeasureName
    )
    # I'll make this pep8 some day, maybe :)
    STEPS = (
        ('FOOD GROUP.csv', CNFFoodGroup, ('id', 'code', 'name', 'name_f')),
        ('FOOD SOURCE.csv', CNFFoodSource, ('id', 'code', 'description', 'description_f')),
        ('FOOD NAME.csv', CNFFoodName, ('id', 'code', 'food_group', 'food_source', 'description', 'description_f', 'date_of_entry', 'date_of_publication', 'country_code', 'scientific_name')),
        ('NUTRIENT AMOUNT.csv', CNFNutrientAmount, ('food', 'nutrient_name', 'nutrient_value', 'standard_error', 'number_of_observations', 'nutrient_source', 'date_of_entry')),
        ('NUTRIENT NAME.csv', CNFNutrientName, ('id', 'nutrient_code', 'nutrient_symbol', 'unit', 'name', 'name_f', 'tagname', 'nutrient_decimals')),
        ('NUTRIENT SOURCE.csv', CNFNutrientSource, ('id', 'code', 'description', 'description_f')),
        ('CONVERSION FACTOR.csv', CNFConversionFactor, ('food', 'measure', 'value', 'date_of_entry')),
        ('MEASURE NAME.csv', CNFMeasureName, ('id', 'name', 'name_f')),
        ('REFUSE AMOUNT.csv', CNFRefuseAmount, ('food', 'refuse_name', 'amount', 'date_of_entry')),
        ('REFUSE NAME.csv', CNFRefuseName, ('id', 'name', 'name_f')),
        ('YIELD AMOUNT.csv', CNFYieldAmount, ('food', 'yield_name', 'amount')),
        ('YIELD NAME.csv', CNFYieldName, ('id', 'name', 'name_f')),
    )
    for filename, model, fields in STEPS:
        print('Importing', filename)
        # TODO Dropping everything probably should be an option
        #      Not sure what to expect when you don't though
        print('  Dropping all items')
        model.objects.all().delete()
        s = time.time()
        cf = csv_reader(filename)
        count = load_data(cf, model, fields, batch=batch)
        d = time.time() - s
        print(' ', count, 'rows imported in %.04fs\n' % d)

In [22]:
source = 'cnf_csv/'
batch = '1000'

run(source, batch)

Importing FOOD GROUP.csv
  Dropping all items
 filepath: cnf_csv/FOOD GROUP.csv
  23 rows imported in 0.0128s

Importing FOOD SOURCE.csv
  Dropping all items
 filepath: cnf_csv/FOOD SOURCE.csv
  16 rows imported in 0.0070s

Importing FOOD NAME.csv
  Dropping all items
 filepath: cnf_csv/FOOD NAME.csv
  5690 rows imported in 2.1198s

Importing NUTRIENT AMOUNT.csv
  Dropping all items
 filepath: cnf_csv/NUTRIENT AMOUNT.csv
  524674 rows imported in 154.1039s

Importing NUTRIENT NAME.csv
  Dropping all items
 filepath: cnf_csv/NUTRIENT NAME.csv
  152 rows imported in 0.0474s

Importing NUTRIENT SOURCE.csv
  Dropping all items
 filepath: cnf_csv/NUTRIENT SOURCE.csv
  20 rows imported in 0.0135s

Importing CONVERSION FACTOR.csv
  Dropping all items
 filepath: cnf_csv/CONVERSION FACTOR.csv
  19505 rows imported in 6.2399s

Importing MEASURE NAME.csv
  Dropping all items
 filepath: cnf_csv/MEASURE NAME.csv
  1162 rows imported in 0.1961s

Importing REFUSE AMOUNT.csv
  Dropping all items
 file

In [20]:
import csv
print(csv.__file__)


/home/nobu/anaconda3/envs/ecowarrior/lib/python3.9/csv.py


In [24]:
# test mongo
q='bean'
foods = CNFFoodName.objects.filter(description__icontains=q) if q else []
#print(foods)
for food in foods:
    print(food.description, type(food))
    

Vegetable oil, soybean <class 'mongo_models.CNFFoodName'>
Vegetable oil, soybean lecithin <class 'mongo_models.CNFFoodName'>
Shortening, household, soybean (hydrogenated) and palm <class 'mongo_models.CNFFoodName'>
Shortening, multipurpose, hydrogenated soybean and palm <class 'mongo_models.CNFFoodName'>
Shortening, industrial, for frying (liquid), soybean <class 'mongo_models.CNFFoodName'>
Shortening, industrial, for frying (solid), soybean <class 'mongo_models.CNFFoodName'>
Shortening, industrial, for baking (cake), soybean <class 'mongo_models.CNFFoodName'>
Shortening, industrial, for baking (pastry), soybean <class 'mongo_models.CNFFoodName'>
Soup, bean with bacon, canned, condensed <class 'mongo_models.CNFFoodName'>
Soup, bean with ham, ready-to-serve <class 'mongo_models.CNFFoodName'>
Soup, bean with bacon, dehydrated <class 'mongo_models.CNFFoodName'>
Soup, bean with bacon, canned, condensed, water added <class 'mongo_models.CNFFoodName'>
Soup, bean with bacon, dehydrated, water